In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_bitcoin as btc
import tasker_link_prediction as t_lp
from splitter import splitter
from models import TwoStream_GCN
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_BTC_OTC.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/BTC_OTC_TwoStream/',
 'classifier_hidden_size': 16,
 'ffn_fusion_size': 8,
 'ffn_hidden_size': 8,
 'gcn_fusion_size': 16,
 'spatial_hidden_size': 16,
 'spatial_input_dim': 64,
 'temporal_hidden_size': 16,
 'temporal_input_dim': 80,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/BTC-OTC/',
 'prep_data_path': 'prep_data/BTC_OTC_neg/'}

In [4]:
data = btc.Bitcoin_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= TwoStream_GCN(spatial_input_dim=config['spatial_input_dim'],
                 temporal_input_dim=config['temporal_input_dim'],
                 spatial_hidden_size=config['spatial_hidden_size'],
                 temporal_hidden_size=config['temporal_hidden_size'],
                 classifier_hidden_size=config['classifier_hidden_size'],
                 gcn_fusion_size=config['gcn_fusion_size'],
                 ffn_fusion_size=config['ffn_fusion_size'],
                 ffn_hiden_size=config['ffn_hidden_size'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])
print(trainer.patience)

Dataset splits sizes:  train 86 dev 14 test 28
10


In [5]:
trainer.train(config['num_epochs'])

Epoch 1, Step 1, Loss: 0.19549354910850525
Epoch 1, Step 2, Loss: 0.15056417882442474
Epoch 1, Step 3, Loss: 0.08804231882095337
Epoch 1, Step 4, Loss: 0.07055313885211945
Epoch 1, Step 5, Loss: 0.04142430052161217
Epoch 1, Step 6, Loss: 0.030554957687854767
Epoch 1, Step 7, Loss: 0.023494461551308632
Epoch 1, Step 8, Loss: 0.026641974225640297
Epoch 1, Step 9, Loss: 0.033453844487667084
Epoch 1, Step 10, Loss: 0.03025112673640251
Epoch 1, Step 11, Loss: 0.02547905594110489
Epoch 1, Step 12, Loss: 0.026202065870165825
Epoch 1, Step 13, Loss: 0.0225321426987648
Epoch 1, Step 14, Loss: 0.02101152203977108
Epoch 1, Step 15, Loss: 0.017704090103507042
Epoch 1, Step 16, Loss: 0.020572448149323463
Epoch 1, Step 17, Loss: 0.03221656754612923
Epoch 1, Step 18, Loss: 0.03895222023129463
Epoch 1, Step 19, Loss: 0.041033729910850525
Epoch 1, Step 20, Loss: 0.03700125217437744
Epoch 1, Step 21, Loss: 0.03457086533308029
Epoch 1, Step 22, Loss: 0.03227464482188225
Epoch 1, Step 23, Loss: 0.02700063

In [6]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.2153835320584412
Validation Metric MAPs: 0.23239875567549842
Test Metric MRRs: 0.26920571903257345
Test Metric MAPs: 0.2872764078421189


(0.26920571903257345, 0.2872764078421189)